In [1]:
%pip install cplex
%pip install docplex

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [16]:
from typing import Tuple, List
import logging

def read_hgr(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Read a DIMACS-like .hgr file and return the number of nodes, edges, and the edge list.

    Args:
        file_path: Path to the .hgr file

    Returns:
        Tuple containing:
        - number of nodes
        - number of edges
        - list of edges, where each edge is a list of vertex indices (1-based)

    Example format:
        c I am a comment
        p hs 6 5
        1 2
        2 3 4
        2 4 5
        1 3 6
        5 6
    """
    num_nodes = 0
    num_edges = 0
    edges = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Parse problem line
            if line.startswith('p'):
                parts = line.split()
                if len(parts) != 4 or parts[1] != 'hs':
                    raise ValueError(f"Invalid problem line format: {line}")
                num_nodes = int(parts[2])
                num_edges = int(parts[3])
                continue

            # Parse edge line
            vertices = [int(v) for v in line.split()]
            if not vertices:
                continue

            # Validate vertex numbers
            if any(v < 1 or v > num_nodes for v in vertices):
                raise ValueError(f"Vertex number out of range in line: {line}")

            edges.append(vertices)

    if not num_nodes or not num_edges:
        raise ValueError("Problem line not found or invalid")

    if len(edges) != num_edges:
        logging.warning(f"Expected {num_edges} edges but found {len(edges)}")

    return num_nodes, num_edges, edges


import docplex.mp.model as cplex

def solve_ilp(num_nodes, num_edges, edges):
    mdl = cplex.Model()

    picked_vertices = mdl.binary_var_list(num_nodes, name="picked_vertices")

    for idx, edge in enumerate(edges):
        mdl.add_constraint(mdl.sum(picked_vertices[node-1] for node in edge) >= 1, "Edge_" + str(idx))

    mdl.minimize(mdl.sum(picked_vertices))



    # print parameters of model
    print(mdl.parameters._params)
    print(mdl.parameters.mip.tolerances.mipgap)
    # mdl.parameters.mip.tolerances.mipgap = 0.2



    solution = mdl.solve()
    mdl.print_information()
    print(mdl.solve_details)
    print(f"Solution size: {solution.objective_value}")
    # print(f"Sol: {solution}")

HGRPATH = "./testset/bremen_subgraph_300.hgr"
if __name__ == "__main__":

    try:
        num_nodes, num_edges, edges = read_hgr(HGRPATH)
        print(f"Graph has {num_nodes} nodes and {num_edges} edges")
        solve_ilp(num_nodes, num_edges, edges)
    except Exception as e:
        print(f"Error reading file: {e}")

Graph has 311 nodes and 309 edges
[docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex.mp.params.StrParameter(parameters.workdir,.), docplex.mp.params.NumParameter(parameters.workmem,2048.0)]
mipgap:number(0.0001)
Model: docplex_model15
 - number of variables: 311
   - binary=311, integer=0